In [1]:
'''
# ----------------------------------------------------------------------
#  Function to save model data regridded to IMERG grid for certain region
# ----------------------------------------------------------------------

'''
    
# == imports ==
# -- packages --
import xarray as xr
import os
import pandas as pd
from easygems import healpix as egh
import healpy as hp
import numpy as np
import pandas as pd

# -- imported scripts --
import os
import sys
sys.path.insert(0, os.getcwd())


/scratch/nf33/public/miniconda/envs/digital_earths_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:

# == post-process ==
def process_data_further(da, process_request):
    ''' Function to save regridded data '''
    var_name, dataset, time_period, t_freq, lon_area, lat_area, resolution, name, _ = process_request
    path_save = f"/scratch/nf33/hk25_DOCmeso/{dataset}_interp/{dataset}_{var_name}_{resolution}_{time_period.replace(' ','')}_{name}.nc"    
    
    try: del da.attrs["hiopy::enable"]
    except: pass

    # Save as netcdf
    #da.to_netcdf(path_save)
    
    #print (f"{dataset} regridded file was saved for {time_period}")
    #del da
    return da

# == pre-process ==
def pre_process(ds, process_request):
    var_name, dataset, time_period, t_freq, lon_area, lat_area, resolution, _, _ = process_request

    # -- pick out variable --    
    da = ds[var_name]#.load()
    
    # -- temporally resample --
    if t_freq == 'hrly':
        da = da.resample(time='1h').mean()
    elif t_freq == '3hrly':
        da = da.resample(time='3h').mean()
    else:
        pass

    # -- select region of interest --
    da = da.sel(lon = slice(int(lon_area.split(':')[0]), int(lon_area.split(':')[1])), 
                lat = slice(int(lat_area.split(':')[0]), int(lat_area.split(':')[1]))
                )
    
    return da

def get_nn_lon_lat_index(nside, lons, lats):
    """
    nside: integer, power of 2. The return of hp.get_nside()
    lons: uniques values of longitudes
    lats: uniques values of latitudes
    returns: array with the HEALPix cells that are closest to the lon/lat grid
    """
    lons2, lats2 = np.meshgrid(lons, lats)
    return xr.DataArray(
        hp.ang2pix(nside, lons2, lats2, nest = True, lonlat = True),
        coords=[("lat", lats), ("lon", lons)],
    )

def crop_region(ds, process_request):
    # Select region of interest (without regridding)
    var_name, dataset, time_period, t_freq, lon_area, lat_area, resolution, _, ds_regrid = process_request
    Slim, Nlim = int(lat_area.split(':')[0]), int(lat_area.split(':')[1])
    Elim, Wlim = int(lon_area.split(':')[0]), int(lon_area.split(':')[1])

    print (Slim, Nlim)
    print (Elim, Wlim)
    # Add lat and lon info, and crop
    ds     = ds.pipe(egh.attach_coords)
    ds_reg = ds.where((ds["lat"] > Slim) & (ds["lat"] < Nlim) & (ds["lon"] > Elim) & (ds["lon"] < Wlim),drop=True)

    print ("region cropped")
    return ds_reg

def regrid(ds, process_request):
    var_name, dataset, time_period, t_freq, lon_area, lat_area, resolution, _, ds_regrid = process_request
    
    # -- pick out variable --    
    da = ds[var_name]#.load()

    # -- temporally resample --
    if t_freq == 'hrly':
        da = da.resample(time='1h').mean()
    elif t_freq == '3hrly':
        da = da.resample(time='3h').mean()
    else:
        pass

    # Find the HEALPix pixels that are closest to the IMERG grid
    lon = ds_regrid['lon'].values
    lat = ds_regrid['lat'].values

    # nside for um simulation, it should be equal to 2**zoom
    this_nside = hp.get_nside(da) # I have to do it to the whole domain
    cells = get_nn_lon_lat_index(this_nside, lon, lat) 

    # Regridded
    ds_regrided = da.isel(cell = cells) # regriding

    return cells, ds_regrided


def get_data(process_request, process_data_further):
    var, dataset, time_str, t_freq, lon_area, lat_area, resolution, _, ds_regrid = process_request

    # -- get file and open data --
    
    if dataset == "IMERG":
        year, month, day = time_str.split('-')
        folder = f'/g/data/ia39/aus-ref-clim-data-nci/gpm/data/V07/{year}'
        if int(month) == 12 and int(day) == 23:                                     # day is missing, give previous days value
            filename = f'3B-HHR.MS.MRG.3IMERG.{year}{month}22.V07A.nc'
            da = xr.open_dataset(f'{folder}/{filename}')
            da['time'] = da['time'] + pd.Timedelta(days=1)
        elif int(month) == 12 and int(day) == 24:                                   # day is missing, give previous days value
            filename = f'3B-HHR.MS.MRG.3IMERG.{year}{month}25.V07A.nc'
            da = xr.open_dataset(f'{folder}/{filename}')
            da['time'] = da['time'] - pd.Timedelta(days=1)
        else:
            filename = f'3B-HHR.MS.MRG.3IMERG.{year}{month}{day}.V07A.nc'
        da = xr.open_dataset(f'{folder}/{filename}')
    
    elif dataset == "UM":
        #folder = f'/scratch/nf33/Healpix_data/{dataset}'
        #filename = f'data.healpix.PT1H.{resolution}.zarr'
        folder = f'/g/data/qx55/uk_node/glm.n2560_RAL3p3/'
        filename = f'data.healpix.PT{t_freq}.{resolution}.zarr'
        da  = xr.open_zarr(f'{folder}/{filename}')
        da  = da.sel(time = time_str) if time_str != "All" else da 
        
    else: # For ICON
        #folder   = f'/scratch/nf33/Healpix_data/{dataset}'
        #filename = f'data.healpix.PT1H.{resolution}.zarr'
        folder = f'/g/data/qx55/germany_node/d3hp003.zarr/'
        filename = f'PT{t_freq}_point_{resolution}_atm.zarr'
        da  = xr.open_zarr(f'{folder}/{filename}')
        da  = da.sel(time = time_str) if time_str != "All" else da 
        
    # -- pre-process --
    if dataset == "IMERG":
        da = pre_process(da, process_request)
        return da

    else:
        #da = crop_region(da, process_request)
        if var == "pr":
            cells_inside, da = regrid(da, process_request)
            return cells_inside, da
        else:
            return da
        #da = process_data_further(da, process_request)
        
        #

In [3]:
def get_model_3d(dataset_, times = "All"): #"UM" or "ICON" time_str: "2020-02-01"
    # exit()

    # Get IMERG data: any date
    var =           'precipitation'
    dataset =       'IMERG'
    time_str =      '2020-03-01'
    t_freq =        'hourly'
    lon_area =      '100:149'
    lat_area =      '-10:10'
    resolution =    0.1

    process_request = [var, dataset, time_str, t_freq, lon_area, lat_area, resolution, None, None]
    ds_imerg = get_data(process_request, process_data_further)
    
    
    print ("IMERG data was read")
    
    # Run precipitation to get grids: one hour
    dataset = dataset_
    var   =         'pr'
    time_str =      '2020-03-01 00:00' # or "All"
    if dataset == "ICON": t_freq = '1H' if var == "pr" else "6H"
    else: t_freq = '1H' if var == "pr" else "3H"
    lon_area =      '100:149'
    lat_area =      '-10:10'
    resolution =    'z9'
    name       =    "MarCont" # Name to add to the saved file
    
    process_request = [var, dataset, time_str, t_freq, lon_area, lat_area, resolution, name, ds_imerg]
    
    if var == "pr":
        cells_inside, ds_model = get_data(process_request, process_data_further)
    else:    
        ds_model = get_data(process_request, process_data_further)
        
    print (f"{dataset} {var} data was read to regrid")

    # Run all other variables in 3D, regrid them after
    var   =         "ta"
    time_str =      times # or "All"
    if dataset == "ICON": t_freq = '1H' if var == "pr" else "6H"
    else: t_freq = '1H' if var == "pr" else "3H"
    
    # Running for one day
    process_request = [var, dataset, time_str, t_freq, lon_area, lat_area, resolution, name, ds_imerg]
    
    if var == "pr":
        cells_inside, ds_model = get_data(process_request, process_data_further)
    else:    
        ds_model = get_data(process_request, process_data_further)

    print (f"{dataset} 3d data was read ")

    # Select some cells
    ds_model = ds_model.isel(cell = cells_inside)

    # Renaming: CF compilant    
    ds_model = ds_model.rename({"lat":"latitude", "lon":"longitude", "pressure":"level"})

    # Add atributes
    ds_model["latitude"].attrs  = {"units": "degrees_north", "long_name":"latitude"}
    ds_model["longitude"].attrs = {"units": "degrees_east", "long_name":"longitude"}

    if dataset == "ICON": 
        ds_model["level"].attrs     = {"units": "Pa", "long_name":"pressure level"}
    elif dataset == "UM":
        ds_model["level"].attrs     = {"units": "hPa", "long_name":"pressure level"}

    # Drop
    ds_model = ds_model.drop_vars("cell")
    ds_model = ds_model.drop_vars("crs")
    
    return ds_model


In [4]:
ds_model = get_model_3d("UM", "2020-02-01")

FileNotFoundError: [Errno 2] No such file or directory: '/g/data/ia39/aus-ref-clim-data-nci/gpm/data/V07/2020/3B-HHR.MS.MRG.3IMERG.20200301.V07A.nc'